# Applying Pre-Implemented Analyses

An *Analysis* class can be applied to a dataset in a flexible manner, such as the parameterisation how and where the data is stored, which derivatives are required, and the computing environment in which to generate the derivatives.

## Inspecting the Analysis Class

We will start by importing a predefined Analysis class from `example.analysis`, which performs the same analysis as the workflow in the [Workflows Notebook](basic_workflow.ipynb). We print the "menu", the list of inputs, derivatives and parameters objects of this class can receive/derive, using the `static_menu` class method.

In [1]:
from example.analysis import BasicBrainAnalysis
print(BasicBrainAnalysis.static_menu())


example.analysis.BasicBrainAnalysis Menu 
-----------------------------------------

Inputs:
    magnitude : nifti_gz
        A magnitude image (e.g. T1w, T2w, etc..)

Derivatives:
    brain : nifti_gz
        Skull-stripped magnitude image
    brain_mask : nifti_gz
        Brain mask used for skull-stripping
    smooth : nifti_gz
        Smoothed magnitude image
    smooth_masked : nifti_gz
        Smoothed and masked magnitude image

Parameters:
    smoothing_fwhm : float
        The full-width-half-maxium radius of the smoothing kernel


## Defining the dataset to analyse

Arcana implicitly handles a lot of the menial tasks involved with data input/outputs such as file format conversions and inserting/retrieving data from a repository service (i.e. XNAT). In defining the dataset to analyse you first define the repository it is stored within and then map elements of the dataset to data menu entries in the Analysis class.